In [1]:
# relative import hack
import sys
sys.path.insert(0, "..")

from covid_dc.school_data import (
    scrape_articles_data,
    scrape_notifications_data,
    parse_incidents,
    data_to_dataframe,
    fix_school_names,
    append_school_levels
)

import pandas as pd

import seaborn as sns

%pylab inline

pd.options.display.max_rows = 250
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = None

Populating the interactive namespace from numpy and matplotlib


In [2]:
incidents = scrape_notifications_data()

In [3]:
incidents = parse_incidents(incidents)

May 10, 2021, notifying them of two positive COVID-19 cases in the building when students were present on May 4 and 5, 2021, respectively.  to  [[], [], [2], [], [], []]
April 1, 2021, notifying them of two positive COVID-19 cases in the building when students were present on March 31 and April 1, 2021, respectively. to  [[], [], [2], [], [], []]


In [4]:
all_data = data_to_dataframe(incidents)

In [5]:
def fix_school_names(all_data: object) -> object:
    name_corrections = {
        "Barnard ES": "Barnard",
        "Beers ES": "Beers",
        "Boone Elementary": "Boone",
        "Bruce-Monrow": "Bruce-Monroe",
        "CW Harris": "C.W. Harris",
        "Excel Academy": "Excel",
        "Excel-Academy": "Excel",
        "GarrisonElementary": "Garrison",
        "H.D.Cooke ": "H.D. Cooke",
        "H.D.Cooke": "H.D. Cooke",
        "HD Cooke": "H.D. Cooke",
        "H.D. Woodson": "Woodson",
        "HD Woodson": "Woodson",
        "Houston ES": "Houston",
        "Ida.B.Wells": "Ida B. Wells",
        "J.O. Wilson Elementary": "J.O. Wilson",
        "Jefferson MS": "Jefferson",
        "John Hayden Johnson Middle School": "Johnson",
        "Kimball Elementary": "Kimball",
        "King ES": "King",
        "King Elementary": "King",
        "Langdon ES": "Langdon",
        "Mann Elementary": "Mann",
        "Maury Elementary": "Maury",
        "McKinley High School": "McKinley High",
        "McKinley Tech High School": "McKinley High",
        "McKinley Middle School": "McKinkey Middle",
        "Miner Elementary": "Miner",
        "Moten Elementary": "Moten",
        "Nalle Elementary School": "Nalle",
        "Oyster-Adams (Adams)": "Oyster-Adams",
        "Peabody and Watkins": "Peabody Watkins",
        "Peabody": "Peabody Watkins",
        "Roosevelt STAY High School": "Roosevelt STAY",
        "School Without Walls at Francis-Stevens": "SWW @ Francis Stevens",
        "School Within Walls": "School Without Walls",
        "Seaton ES": "Seaton",
        "Stuart Hobson": "Stuart-Hobson",
        "Stuart-Hobson Middle": "Stuart-Hobson",
        "Thomas Elementary": "Thomas",
        "Thomson Elementary": "Thomson",
        "Tubman Elementary School": "Tubman",
        "Van Ness Elementary School": "Van Ness",
    }
    all_data["school"] = all_data["school"].replace(name_corrections)
    return all_data

In [6]:
all_data = fix_school_names(all_data)

In [7]:
all_data = append_school_levels(all_data)

In [9]:
# just copypastaing this to gsheets for now
all_data[all_data.incident_date >= "2021-08-20"][
    ["incident_date", "letter_date", "school", "cases_count", "school_level", "incident_text"]
].to_clipboard(index=False)

In [10]:
# Dropping sample data in a folder in the repo
all_data[all_data.incident_date >= "2021-08-20"].to_csv(
    "../data/school_covid_data_2021_2021_school_year.csv", index=False, ifexists
)

## Below here is workspace